In [1]:
# Cell 1
import os, glob, pickle
import numpy as np
import pandas as pd
from pathlib import Path

PROJECT_ROOT = r"C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease"
DATA_ROOT = Path(PROJECT_ROOT) / "data" / "PaHaW_dataset"
PAHAW_PUBLIC = DATA_ROOT / "PaHaW_public"
METADATA_FILE = DATA_ROOT / "PaHaW_files" / "corpus_PaHaW.xlsx"

meta = pd.read_excel(METADATA_FILE)
meta['ID'] = meta['ID'].astype(str).str.zfill(5)
print("Metadata loaded:", meta.shape)


Metadata loaded: (75, 10)


In [2]:
# Cell 2
# If you ran notebook 03 in same kernel and have subs, beta_df, ellipse_df, use them.
# Otherwise re-run notebook 03 (Cells 0..C) to get these variables, or load pickles if you saved them.

vars_present = {v: (v in globals()) for v in ['subs','beta_df','ellipse_df','df_proc']}
print("Present in kernel:", vars_present)
# If any False, re-run notebook 03 or define them here.


Present in kernel: {'subs': False, 'beta_df': False, 'ellipse_df': False, 'df_proc': False}


In [4]:
import pickle
import pandas as pd
import os

DATA_DIR = r"C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\data\processed"

df_proc = pd.read_pickle(os.path.join(DATA_DIR, "df_proc.pkl"))
beta_df = pd.read_pickle(os.path.join(DATA_DIR, "beta_df.pkl"))
ellipse_df = pd.read_pickle(os.path.join(DATA_DIR, "ellipse_df.pkl"))

with open(os.path.join(DATA_DIR, "subs.pkl"), "rb") as f:
    subs = pickle.load(f)

print("Loaded df_proc:", df_proc.shape)
print("Loaded beta_df:", beta_df.shape)
print("Loaded ellipse_df:", ellipse_df.shape)
print("Loaded subs:", len(subs))


Loaded df_proc: (1772, 18)
Loaded beta_df: (176, 5)
Loaded ellipse_df: (176, 5)
Loaded subs: 176


In [5]:
# Cell A — build per-substroke features (for the sample loaded df_proc, subs, beta_df, ellipse_df)
import numpy as np
import pandas as pd

rows = []
for idx, (s,e) in enumerate(subs):
    seg = df_proc.iloc[s:e+1].reset_index(drop=True)
    duration = seg['time_s'].iloc[-1] - seg['time_s'].iloc[0]
    amp_x = seg['x_s'].max() - seg['x_s'].min()
    amp_y = seg['y_s'].max() - seg['y_s'].min()
    amplitude = np.sqrt(amp_x**2 + amp_y**2)
    mean_speed = seg['speed'].mean()
    max_speed = seg['speed'].max()
    mean_press = seg['press'].mean()
    std_press = seg['press'].std()
    mean_curv = seg.get('curvature', pd.Series([0]*len(seg))).mean()
    beta_row = beta_df.loc[idx, 'beta'] if 'beta' in beta_df.columns else None
    ellipse_row = ellipse_df.loc[idx, 'ellipse'] if 'ellipse' in ellipse_df.columns else None
    rows.append({
        "sub_id": idx, "start": s, "end": e, "n_points": len(seg),
        "duration": duration, "amplitude": amplitude,
        "mean_speed": mean_speed, "max_speed": max_speed,
        "mean_press": mean_press, "std_press": std_press,
        "mean_curvature": mean_curv,
        "beta": beta_row, "ellipse": ellipse_row
    })

features_df = pd.DataFrame(rows)
print("features_df shape:", features_df.shape)
features_df.head(8)


features_df shape: (176, 13)


,sub_id,start,end,n_points,duration,amplitude,mean_speed,max_speed,mean_press,std_press,mean_curvature,beta,ellipse
0,0,0,8,9,60.0,8.500000,0.130762,0.279508,255.333333,118.844436,0.0,"{'A': 0.45546074676128684, 't0': 0.49999999999...","{'xc': 2083.588512327638, 'yc': -14.9097526102..."
1,1,9,31,23,166.0,57.923321,0.351282,0.557367,594.739130,41.231296,0.0,"{'A': 0.809963140150684, 't0': 67.499999999999...","{'xc': 4166.763479231354, 'yc': -55.6745831704..."
2,2,32,43,12,83.0,52.642399,0.625454,0.775182,594.666667,17.327505,0.0,"{'A': 0.8298528784753406, 't0': 239.9000000000...","{'xc': 2068.4835930239747, 'yc': -41.428660770..."
3,3,44,53,10,68.0,43.678116,0.653507,0.809524,604.400000,10.458383,0.0,"{'A': 0.8925359033501874, 't0': 329.9000000000...","{'xc': 4056.202814178768, 'yc': -98.0196536529..."
4,4,54,74,21,151.0,105.338080,0.722258,1.022424,667.952381,24.622096,0.0,"{'A': 1.0685985941632399, 't0': 404.9000000000...","{'xc': 4001.436020735016, 'yc': -117.747384150..."
5,5,75,93,19,136.0,118.734063,0.870493,1.051939,744.894737,12.119290,0.0,"{'A': 1.1131331222505965, 't0': 562.9000000000...","{'xc': 3653.2712529597884, 'yc': 45.0913760975..."
6,6,94,105,12,83.0,74.276510,0.925731,1.110658,788.666667,23.891167,0.0,"{'A': 1.1808897491992003, 't0': 705.9000000000...","{'xc': 3891.078049601283, 'yc': 91.75719012325..."
7,7,106,113,8,53.0,63.085480,1.166993,1.415816,832.000000,10.141851,0.0,"{'A': 1.4763502933468409, 't0': 795.9000000000...","{'xc': 4077.105024296492, 'yc': 248.2189827411..."


In [6]:
# Cell B — unpack nested dict columns to numeric columns
def unpack_dict_col(df, colname, prefix):
    keys = set()
    for v in df[colname].dropna():
        if isinstance(v, dict):
            keys.update(v.keys())
    keys = sorted(keys)
    for k in keys:
        df[f"{prefix}_{k}"] = df[colname].apply(lambda d: d.get(k) if isinstance(d, dict) else np.nan)
    return df

features_df = unpack_dict_col(features_df, "beta", "beta")
features_df = unpack_dict_col(features_df, "ellipse", "ell")

# quick sanity print
num_beta = features_df['beta'].notnull().sum()
num_ell  = features_df['ellipse'].notnull().sum()
print("Rows:", len(features_df), "beta dicts:", num_beta, "ellipse dicts:", num_ell)
display(features_df[['sub_id','n_points','duration','amplitude','mean_speed','beta_A','beta_a','beta_b','ell_a','ell_b','ell_ecc']].head(6))


Rows: 176 beta dicts: 176 ellipse dicts: 168


,sub_id,n_points,duration,amplitude,mean_speed,beta_A,beta_a,beta_b,ell_a,ell_b,ell_ecc
0,0,9,60.0,8.500000,0.130762,0.455461,0.320931,0.996356,NaN,8116.006576,0.0
1,1,23,166.0,57.923321,0.351282,0.809963,0.835436,0.149417,NaN,710.737258,0.0
2,2,12,83.0,52.642399,0.625454,0.829853,0.155913,0.100000,NaN,3211.576897,0.0
3,3,10,68.0,43.678116,0.653507,0.892536,0.100000,0.187103,NaN,4338.461779,0.0
4,4,21,151.0,105.338080,0.722258,1.068599,0.306295,0.100000,NaN,1408.147561,0.0
5,5,19,136.0,118.734063,0.870493,1.113133,0.134684,0.100000,2281.76872,NaN,0.0


In [7]:
# Cell C — mark success flags and save CSV
features_df['ell_success'] = ~features_df['ell_xc'].isnull()
features_df['beta_success'] = ~features_df['beta_A'].isnull()
print("Ellipse success:", int(features_df['ell_success'].sum()), "/", len(features_df))
print("Beta success:", int(features_df['beta_success'].sum()), "/", len(features_df))

# save CSV
out_csv = r"C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\sample_substroke_features.csv"
features_df.to_csv(out_csv, index=False)
print("Saved:", out_csv)


Ellipse success: 168 / 176
Beta success: 176 / 176
Saved: C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\sample_substroke_features.csv


In [9]:
# Cell D — compute simple fuzzy memberships (speed, pressure, curvature)
import numpy as np
import skfuzzy as fuzz

# build universes (handle flat ranges)
speed_vals = features_df['mean_speed'].fillna(0).to_numpy()
press_vals = features_df['mean_press'].fillna(0).to_numpy()
curv_vals  = features_df['mean_curvature'].fillna(0).to_numpy()

def safe_universe(arr):
    lo, hi = arr.min(), arr.max()
    if hi == lo:
        hi = lo + 1e-3
    return np.linspace(lo, hi, 100)

u_speed = safe_universe(speed_vals)
u_press = safe_universe(press_vals)
u_curv  = safe_universe(curv_vals)

speed_low = fuzz.trapmf(u_speed, [u_speed[0], u_speed[0], u_speed[int(25)], u_speed[int(40)]])
speed_med = fuzz.trimf(u_speed, [u_speed[int(30)], u_speed[int(50)], u_speed[int(70)]])
speed_high= fuzz.trapmf(u_speed, [u_speed[int(60)], u_speed[int(75)], u_speed[-1], u_speed[-1]])

press_low = fuzz.trapmf(u_press, [u_press[0], u_press[0], u_press[int(25)], u_press[int(40)]])
press_med = fuzz.trimf(u_press, [u_press[int(30)], u_press[int(50)], u_press[int(70)]])
press_high= fuzz.trapmf(u_press, [u_press[int(60)], u_press[int(75)], u_press[-1], u_press[-1]])

curv_low = fuzz.trapmf(u_curv, [u_curv[0], u_curv[0], u_curv[int(25)], u_curv[int(40)]])
curv_med = fuzz.trimf(u_curv, [u_curv[int(30)], u_curv[int(50)], u_curv[int(70)]])
curv_high= fuzz.trapmf(u_curv, [u_curv[int(60)], u_curv[int(75)], u_curv[-1], u_curv[-1]])

def fuzz_membership(value, universe, low, med, high):
    m_low = np.interp(value, universe, low)
    m_med = np.interp(value, universe, med)
    m_high= np.interp(value, universe, high)
    return m_low, m_med, m_high

for i, row in features_df.iterrows():
    sp = row['mean_speed']
    pr = row['mean_press']
    cv = row['mean_curvature']
    ms = fuzz_membership(sp, u_speed, speed_low, speed_med, speed_high)
    mp = fuzz_membership(pr, u_press, press_low, press_med, press_high)
    mc = fuzz_membership(cv, u_curv, curv_low, curv_med, curv_high)
    features_df.loc[i, 'f_speed_low']  = ms[0]
    features_df.loc[i, 'f_speed_med']  = ms[1]
    features_df.loc[i, 'f_speed_high'] = ms[2]
    features_df.loc[i, 'f_press_low']  = mp[0]
    features_df.loc[i, 'f_press_med']  = mp[1]
    features_df.loc[i, 'f_press_high'] = mp[2]
    features_df.loc[i, 'f_curv_low']   = mc[0]
    features_df.loc[i, 'f_curv_med']   = mc[1]
    features_df.loc[i, 'f_curv_high']  = mc[2]

print("Fuzzy features added. Example:")
display(features_df[['sub_id','mean_speed','f_speed_low','f_speed_med','f_speed_high','mean_press','f_press_low','f_press_med']].head(6))


Fuzzy features added. Example:


,sub_id,mean_speed,f_speed_low,f_speed_med,f_speed_high,mean_press,f_press_low,f_press_med
0,0,0.130762,1.0,0.0,0.0,255.333333,1.0,0.000000
1,1,0.351282,1.0,0.0,0.0,594.739130,0.0,0.787396
2,2,0.625454,1.0,0.0,0.0,594.666667,0.0,0.787069
3,3,0.653507,1.0,0.0,0.0,604.400000,0.0,0.831005
4,4,0.722258,1.0,0.0,0.0,667.952381,0.0,0.882123
5,5,0.870493,1.0,0.0,0.0,744.894737,0.0,0.534809
